In [1]:
import re
import heapq 
import copy
import sys
import math

sys.setrecursionlimit(100000)

In [2]:
x1=math.radians(30)

In [3]:
1 - math.sin(math.radians(30))

0.5

In [4]:
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 1.5, 1.866),
    (60,180) : (1, 1.866, 1.5),
    (90,180) : (1, 2, 1),
    (120,180) : (1, 1.866, 0.5),
    (150,180) : (1, 1.5, 0.144),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 0.5, 0.144),
    (120,0) : (1, 0.144, 0.5),
    (90,0) : (1, 0, 1),
    (60,0) : (1, 0.144, 1.5),
    (30,0) : (1, 0.5, 1.866),
    
    #(0,0) : (1,1,2)
    (30,270) : (1.5, 1, 1.866),
    (60,270) : (1.866, 1, 1.5),
    (90,270) : (2, 1, 1),
    (120,270) : (1.866, 1, 0.5),
    (150,270) : (1.5, 1, 0.144),
    #(180,180) : (1,1,0)
    (150,90) : (0.5, 1, 0.144),
    (120,90) : (0.144, 1, 0.866),
    (90,90) : (0, 1, 1),
    (60,90) : (0.144, 1, 1.5),
    (30,90) : (0.5, 1, 1.866),
    
    #(90,0) : (1,2,1)
    (90,330) : (1.5, 1.866, 1),
    (90,300) : (1.866, 1.5, 1),
    #(90,270) : (2,1,1)
    (90,240) : (1.866, 0.866, 1),
    (90,210) : (1.5, 0.5, 1),
    #(90,180) : (1,0,1)
    (90,150) : (0.5, 0.5, 1),
    (90,120) : (0.144, 0.866, 1),
    #(90,90) : (0,1,1)
    (90,60) : (0.144, 1.5, 1),
    (90,30) : (0.5, 1.866, 1)
    
}

axis0_180Base=[(0, 0),(30, 0),(60, 0),(90, 0),(120, 0),(150, 0),(180, 180),(150, 180),(120, 180),(90, 180),(60, 180),(30, 180)]

axis90_270Base=[(0, 0),(30, 90),(60, 90),(90, 90),(120, 90),(150, 90),(180, 180),(150, 270),(120, 270),(90, 270),(60, 270),(30, 270)]

equatorBase = [(90,0),(90,30),(90,60),(90,90),(90,120),(90,150),(90,180),(90,210),(90,240),(90,270),(90,300),(90,330)]

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance

    

In [5]:
heuristic((0,0), (180,180))

4.0

In [6]:
axis0_180 = {}
axis90_270 = {}
equator = {}

def populateAxes(tileNums):
    actualPos = (int(tileNums[0]), int(tileNums[1]))
    truePos = (int(tileNums[2]), int(tileNums[3]))
                
    #populate equator
    if (actualPos[0] == 90):
        equator[actualPos] = truePos
        
    #populate axis0-180
    if (actualPos[1] == 0):
        axis0_180[actualPos] = truePos
    if (actualPos[1] == 180):
        axis0_180[actualPos] = truePos
                
    #populate axis90-270
    if(actualPos[1] == 90):
        axis90_270[actualPos] = truePos
    if(actualPos[1] == 270):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 0 & actualPos[1] == 0):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 180 & actualPos[1] == 180):
        axis90_270[actualPos] = truePos
    
def readState(filename):
    file1 = open(filename,"r")
    lines = file1.readlines()
    #print (lines[1:31])
    #print("\n")
        
    for line in lines[1:31]:
        tileNums = re.findall(r'\d+', line)
        #print(tileNums)
        if(len(tileNums)>4):
            #print(tileNums[2:6])
            populateAxes(tileNums[2:6])
        else:
            #print(tileNums)
            populateAxes(tileNums)

readState("Puzzle2-2.mb")

In [7]:
class GlobeState:
    
    def __init__(self, axis0_180, axis90_270, equator, g):
        self.axis0_180 = axis0_180
        self.axis90_270 = axis90_270
        self.equator = equator
        self.g = g
        
    #rotate clockwise axis 0-180
    def RC_axis0_180(self):
        newD = {}
        
        newD[(30,180)] = self.axis0_180[(0,0)]
        
        for i in [60, 90, 120, 150, 180]:
            newD[(i, 180)] = self.axis0_180[(i-30,180)]
        
        newD[(150,0)] = self.axis0_180[(180,180)]
        
        for i in [120, 90, 60, 30, 0]:
            newD[(i, 0)] = self.axis0_180[(i+30,0)]
        
        #swaps in axis90_270    
        self.axis90_270[(0,0)] = self.axis0_180[(30,0)]
        self.axis90_270[(180,180)] = self.axis0_180[150,180]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[120,0]
        self.equator[(90,180)] = self.axis0_180[60,180]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RCC_axis0_180(self):
        newD = {}
        
        newD[(0,0)] = self.axis0_180[(30,180)]
        
        for i in [30, 60, 90, 120, 150]:
            newD[(i,0)] = self.axis0_180[(i-30,0)]
        
        newD[(180, 180)] = self.axis0_180[(150,0)]
        
        for i in [150, 120, 90, 60, 30]:
            newD[(i,180)] = self.axis0_180[(i+30, 180)]
            
        #swaps in axis90_270
        self.axis90_270[(0,0)] = self.axis0_180[(30,180)]
        self.axis90_270[(180, 180)] = self.axis0_180[(150, 0)]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[(60,0)]
        self.equator[(90,180)] = self.axis0_180[(120, 180)]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RC_axis90_270(self):
        newD = {}
        
        newD[(30,270)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,270)] = self.axis90_270[(i-30, 270)]
        
        newD[(180,180)] = self.axis90_270[(150,270)]
        newD[(150, 90)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i, 90)] = self.axis90_270[(i+30,90)]
        
        newD[(0,0)] = self.axis90_270[(30,90)]
        
        #swaps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30,90)]
        self.axis0_180[(180,180)] = self.axis90_270[(150,270)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(120,90)]
        self.equator[(90,270)] = self.axis90_270[(60,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RCC_axis90_270(self):
        newD = {}
        
        newD[(0,0)] = self.axis90_270[(30, 270)]
        newD[(30,90)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,90)] = self.axis90_270[(i-30,90)]
            
        newD[(180,180)] = self.axis90_270[(150,90)]
        newD[(150, 270)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i,270)] = self.axis90_270[(i+30, 270)]
        
        #sawps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30, 270)]
        self.axis0_180[(180,180)] = self.axis90_270[(150, 90)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(60,90)]
        self.equator[(90,270)] = self.axis90_270[(120,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RC_equator(self):
        newD = {}
        
        newD[(90,330)] = self.equator[(90,0)]
        
        for i in [300, 270, 240, 210, 180, 150, 120, 90, 60, 30, 0]:
            newD[(90, i)] = self.equator[(90, i+30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 30)]
        self.axis0_180[(90,180)] = self.equator[(90, 210)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 120)]
        self.axis90_270[(90, 270)] = self.equator[(90, 300)]
        
        #new equator
        self.equator = newD
        
    def RCC_equator(self):
        newD = {}
        
        newD[(90,0)] = self.equator[(90,330)]
        
        for i in [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]:
            newD[(90, i)] = self.equator[(90, i-30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 330)]
        self.axis0_180[(90,180)] = self.equator[(90, 150)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 60)]
        self.axis90_270[(90, 270)] = self.equator[(90, 240)]
        
        #new equator
        self.equator = newD
        
    def getCurrentHeuristic(self):
        H = 0
        
        for key in self.axis0_180.keys():
            H = H + heuristic(key, self.axis0_180[key])
        
        for key in self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.axis90_270[key])
        
        for key in self.equator.keys() - self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.equator[key])
        
        return H/12
    
    
    def isSolved(self):
        flag = True
        for key in self.axis0_180.keys():
            if (key != self.axis0_180[key]):
                flag = False
        for key in self.axis90_270.keys():
            if (key != self.axis90_270[key]):
                flag = False
        for key in self.equator.keys():
            if (key != self.equator[key]):
                flag = False
        
        return flag
    
    

In [8]:
gs = GlobeState(axis0_180, axis90_270, equator, 0)
#gs.RCC_axis0_180()

gs.isSolved()

False

In [9]:
gs1 = copy.deepcopy(gs)
gs1.equator == gs.equator

True

In [10]:
def copyState(state):
    newState = GlobeState(state.axis0_180, state.axis90_270, state.equator, state.g)
    return newState

In [11]:
def Astar(n):
    #initialize open and closed lists
    openList=set()
    closedList=set()
    
    #k=0
    #add start state to openList
    openList.add(gs)
    
    while(len(openList)!=0):
        #print(closedList)
        #k=k+1
        #find state with min f val
        minVal=10000000000
        for state in openList:
            if(state.getCurrentHeuristic() + state.g < minVal):
                minVal = state.getCurrentHeuristic() + state.g
                q = state
        print (minVal)
        
        #q.g=q.g+(2*(math.pi)/12)
        
        #create successors
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
        
        #for each succesor, if successor in either list and f val in listed item is greater, skip this successor.
        #else add it to openList
        for state in [gs1, gs2, gs3, gs4, gs5, gs6]:
            if(state.isSolved()):
                return state
            
            state.g = q.g + (2*(math.pi)/12)
            f = state.g + state.getCurrentHeuristic()
            
            flag = True 
            
            if(state.g > n):
                flag=False
                
            for stateX in closedList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    #if stateX.getCurrentHeuristic() + stateX.g < f:
                    flag = False
                    print("YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
            
            for stateX in openList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    if state.g < stateX.g:
                        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
                        openList.remove(stateX)
                        openList.add(state)
                    flag = False
            
            if(flag==True):
                openList.add(state)
        
        #remove q from openList
        print(len(openList))
        openList.remove(q)
        closedList.add(q)
        
        
        

In [12]:
gh = Astar(2000)

5.339238349589787
7
5.134131886253394
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11
5.227366021681467
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
15
5.016054368672338
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
19
4.6792768484402
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
23
5.084666177432169
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
27
4.783963163728036
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
31
5.126290761844845
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
35
5.170823957058021
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
39
4.825069855608307
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
43
5.2609683229660895
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
47
5.294168901561001
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
51
4.5614925301426625
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
55
4.052742491380616
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
59
4.93953477425354
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
63
5.3178826449630

5.937034510877299
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
499
5.959949901958559
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
503
5.976881843507369
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
507
5.979203380141708
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
511
5.986432705942228
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
515
5.993999692468437
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
519
6.015504810283466
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
523
6.024489716671337
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
527
6.0344052998616755
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
531
5.679771172554215
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
535
6.034938204514976
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
539
5.689184103065263
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
543
6.036243117866418
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
547
5.681608990558956
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
551
6.045383676055744
Y

6.315787084675147
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
974
6.25078633136653
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
978
6.3166831045957625
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
982
6.317220798557684
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
986
6.317986874839688
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
990
6.3189520301376625
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
994
6.323903275686977
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
998
6.328337794952095
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1002
6.329780304645361
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1006
6.332931677383354
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1010
6.333366850737186
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1014
6.3377723843316565
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1018
6.3396944010638965
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1022
6.038991387359765
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1026
6.34014912

1437
6.455125879955961
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1441
6.4551406786784105
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1445
6.345088837624212
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1449
6.453958881330795
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1453
6.455307025164105
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1457
6.455527458221287
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1461
6.45862094768267
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1465
6.376787003413712
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1469
6.459193673520498
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1473
6.459598032617457
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1477
6.459699361678099
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1481
6.4598997057611465
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1485
6.4602549883892575
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1489
6.120143151477161
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1493

YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1888
6.583308636192084
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1891
6.584544282149667
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1895
6.584747192496271
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1899
6.586318923916972
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1903
6.249541403684833
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1907
6.5867137565462475
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1911
5.615921556026143
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1915
5.221139019146737


In [13]:
gh.g

4.1887902047863905

In [2]:
int("inf")

ValueError: invalid literal for int() with base 10: 'inf'

In [7]:
def Astar(gs):
    k=0;
    h = []
    heapq.heappush(h, (gs.getCurrentHeuristic(), gs))
    
    while(h[0][1].isSolved() == False):    
        gs = heapq.heappop(h)[1]
        k=k+1
        print(k)
        gs1 = copy.copy(gs)
        gs2 = copy.copy(gs)
        gs3 = copy.copy(gs)
        gs4 = copy.copy(gs)
        gs5 = copy.copy(gs)
        gs6 = copy.copy(gs)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
             
        a = gs1.getCurrentHeuristic()+k
        b = gs2.getCurrentHeuristic()+k
        c = gs3.getCurrentHeuristic()+k
        d = gs4.getCurrentHeuristic()+k
        e = gs5.getCurrentHeuristic()+k
        f = gs6.getCurrentHeuristic()+k
        
        if ((a,gs1) not in h):
            heapq.heappush(h, (a, gs1))
        if ((b,gs2) not in h):
            heapq.heappush(h, (b, gs2))
        if ((c,gs3) not in h):
            heapq.heappush(h, (c, gs3))
        if ((d,gs4) not in h):
            heapq.heappush(h, (d, gs4))
        if ((e,gs5) not in h):
            heapq.heappush(h, (e, gs5))
        if ((f,gs6) not in h):
            heapq.heappush(h, (f, gs6))
        #print(type(d))
    return k

In [ ]:

    
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 0.67, 1.67),
    (60,180) : (1, 0.33, 1.33),
    (90,180) : (1, 0, 1),
    (120,180) : (1, 0.33, 0.67),
    (150,180) : (1, 0.67, 0.33),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 1.33, 0.33),
    (120,0) : (1, 1.67, 0.67),
    (90,0) : (1, 2, 1),
    (60,0) : (1, 1.67, 1.33),
    (30,0) : (1, 1.33, 1.67),
    
    (30,270) : (1.33, 1, 1.67),
    (60,270) : (1.67, 1, 1.33),
    (90,270) : (2, 1, 1),
    (120,270) : (1.67, 1, 0.67),
    (150,270) : (1.33, 1, 0.33),
    (150,90) : (0.67, 1, 0.33),
    (120,90) : (0.33, 1, 0.67),
    (90,90) : (0, 1, 1),
    (60,90) : (0.33, 1, 1.33),
    (30,90) : (0.67, 1, 1.67),
    
    (90,330) : (1.33, 1.67, 1),
    (90,300) : (1.67, 1.33, 1),
    (90,240) : (1.67, 0.67, 1),
    (90,210) : (1.33, 0.33, 1),
    (90,150) : (0.67, 0.33, 1),
    (90,120) : (0.33, 0.67, 1),
    (90,60) : (0.33, 1.33, 1),
    (90,30) : (0.67, 1.67, 1)
    
}

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance

    